In [62]:
import random
import json

class game_main_menu:
    def main_menu():
        print("MAIN MENU:")
        print("1. Create New Game")
        print("2. Load Saved Game")
        print("3. About")
        print("4. Exit")
        
    def create_new_game():
        print("Creating New Game")
        print("Please Enter your name:")
        name = input("> ")
        print("> " + name)
        print("Enter your age: ")
        age = input("> ")
        print("> " + age)
        print("Hello, " + name + "! Welcome to the game.")
        print("You are lost in the forest with only a knife.")
        
        player = Player(name=name, weapon="Knife", health=100, stength=10, defense=5, xp=0)
        Game().start(player)
        player.choose_inventory()
        player.save_game()
        return player
    
    def load_saved_game():
        try:
            with open("save_game.json", "r") as file:
                data = json.load(file)
                player = Player(
                    name=data['name'],
                    age=data['age'],
                    weapon=data['weapon'],
                    health=data['health'],
                    strength=data['strength'],
                    defense=data['defense'],
                    xp=data['xp']
                )
                player.inventory.potions = data['inventory']['potions']
                player.inventory.attack_boost = data['inventory']['attack_boost']
                player.inventory.defense_boost = data['inventory']['defense_boost']
                print(f"Game Loaded! Welcome back, {player.name}.")
                player.inventory.show_inventory()
                return player
        except FileNotFoundError:
            print("No saved game found.")
            return None
        
    def main():
        game_main_menu.main_menu()
        choice = input("> ")

        if choice == '1':
            print("> " + choice)
            game_main_menu.create_new_game()
        elif choice == '2':
            player = game_main_menu.load_saved_game()
            if player:
                print(f"Loaded {player.name}'s game.")
        elif choice == '3':
            print("About - This is a simple text adventure game.")
        elif choice == '4':
            print("Exiting game. Goodbye!")
        else:
            print("Invalid choice. Please select a valid option.")



class Entity:
    def __init__(self, name, health, strength, defense):
        self.name = name
        self.health = health
        self.strength = strength
        self.defense = defense
    
    
    
    
class Player(Entity):
    def __init__(self, name, weapon, health, strength, defense, xp):
        super().__init__(name, health, strength, defense)
        self.weapon = weapon
        self.xp = xp
        self.level = 1
        self.inventory = Inventory()

    def level_up(self):
        required_xp = self.level * 100
        if self.xp >= required_xp:
            self.level += 1
            self.health += 20
            self.strength += 5
            self.defense += 3
            self.xp -= required_xp
            print(f"{self.name} leveled up to Level {self.level}!")
    

    def choose_inventory(self):
        """Permet au joueur de choisir ce qu'il veut dans son inventaire"""
        print("Choose items for your inventory:")
        
        while True:
            print("\n1. Add Potion\n2. Add Attack Boost\n3. Add Defense Boost\n4. Show Inventory\n5. Exit")
            choice = input("> ")

            if choice == '1':
                amount = int(input("How many potions? > "))
                self.inventory.buy_potions(amount)
            elif choice == '2':
                amount = int(input("How many attack boosts? > "))
                self.inventory.buy_attack_boost(amount)
            elif choice == '3':
                amount = int(input("How many defense boosts? > "))
                self.inventory.buy_defense_boost(amount)
            elif choice == '4':
                self.inventory.show_inventory()
            elif choice == '5':
                print("Exiting inventory selection.")
                break
            else:
                print("Invalid choice. Please select a valid option.")

    
    def attack(self, enemy):
        damage = max(0, self.strength - enemy.defense)
        print(f"{self.name} attacks {enemy.name} with {self.weapon} and causes {damage} damage.")
        return damage
    
    def take_damage(self, damage):
        self.health -= damage
        if self.health <= 0:
            print(f"{self.name} has been defeated.")
            return False
        return True

    def use_potion(self):
        if self.inventory.potions > 0:
            self.health += 30
            self.inventory.potions -= 1
            print(f"{self.name} used a potion. Health is now {self.health}.")
        else:
            print("No potions left!")
    
    def gain_xp(self, amount):
        self.xp += amount
        print(f"{self.name} gained {amount} XP.")
        self.level_up()
        
        
    def save_game(self):
        """Sauvegarder les informations du joueur dans un fichier JSON"""
        data = {
            'name': self.name,
            'age': self.age,
            'weapon': self.weapon,
            'health': self.health,
            'strength': self.strength,
            'defense': self.defense,
            'inventory': {
                'potions': self.inventory.potions,
                'attack_boost': self.inventory.attack_boost,
                'defense_boost': self.inventory.defense_boost
            }
        }
        with open("savegame.json", "w") as file:
            json.dump(data, file)
        print("Game saved successfully.")
        



class Enemy(Entity):
    def __init__(self, name, level, health, strength, defense):
        super().__init__(name, health, strength, defense)
        self.level = level

    def attack(self, player):
        damage = max(0, self.strength - player.defense)
        print(f"{self.name} attacks {player.name} and causes {damage} damage.")
        return damage
    


class Inventory:
    def __init__(self, potions=0, attack_boost=0, defense_boost=0):
        self.potions = potions
        self.attack_boost = attack_boost
        self.defense_boost = defense_boost

    def buy_potions(self, amount, gold):
        """Acheter des potions si suffisamment d'or disponible"""
        potion_cost = 10
        total_cost = amount * potion_cost

        if gold >= total_cost:
            self.potions += amount
            gold -= total_cost
            print(f"{amount} potion(s) achetée(s). Total : {self.potions}. Il vous reste {gold} pièces d'or.")
        else:
            print("Vous n'avez pas assez d'or pour acheter ces potions.")
        return gold

    def use_potion(self):
        """Utiliser une potion si disponible"""
        if self.potions > 0:
            self.potions -= 1
            print(f"Potion utilisée. Il reste {self.potions} potion(s).")
        else:
            print("Aucune potion disponible.")

    def buy_attack_boost(self, amount, gold):
        """Acheter des boosts d'attaque si suffisamment d'or disponible"""
        boost_cost = 20
        total_cost = amount * boost_cost

        if gold >= total_cost:
            self.attack_boost += amount
            gold -= total_cost
            print(f"{amount} boost(s) d'attaque acheté(s). Total : {self.attack_boost}. Il vous reste {gold} pièces d'or.")
        else:
            print("Vous n'avez pas assez d'or pour acheter ces boosts d'attaque.")
        return gold

    def use_attack_boost(self):
        """Utiliser un boost d'attaque si disponible"""
        if self.attack_boost > 0:
            self.attack_boost -= 1
            print(f"Boost d'attaque utilisé. Il reste {self.attack_boost} boost(s) d'attaque.")
        else:
            print("Aucun boost d'attaque disponible.")

    def buy_defense_boost(self, amount, gold):
        """Acheter des boosts de défense si suffisamment d'or disponible"""
        boost_cost = 20
        total_cost = amount * boost_cost

        if gold >= total_cost:
            self.defense_boost += amount
            gold -= total_cost
            print(f"{amount} boost(s) de défense acheté(s). Total : {self.defense_boost}. Il vous reste {gold} pièces d'or.")
        else:
            print("Vous n'avez pas assez d'or pour acheter ces boosts de défense.")
        return gold

    def use_defense_boost(self):
        """Utiliser un boost de défense si disponible"""
        if self.defense_boost > 0:
            self.defense_boost -= 1
            print(f"Boost de défense utilisé. Il reste {self.defense_boost} boost(s) de défense.")
        else:
            print("Aucun boost de défense disponible.")

    def show_inventory(self):
        """Afficher l'état de l'inventaire"""
        print(f"Inventaire : {self.potions} potion(s), {self.attack_boost} boost(s) d'attaque, {self.defense_boost} boost(s) de défense.")       
        
        
class Game:
    def __init__(self):
        self.map = {
            (0, 0): "Starting point",
            (1, 0): "A calm clearing",
            (0, 1): "A dark cave",
            (-1, 0): "A mysterious river",
            (0, -1): "A thick forest",
            (2, 0): "Boss Lair"
        }
        self.boss_location = (2, 0)
    
    def start(self, player):
        print("Game started!")
        current_location = (0, 0)
        while player.health > 0:
            print(f"Current location: {self.map.get(current_location, 'Unknown area')}")
            move = input("Where would you like to go? (Go North, Go South, Go East, Go West): ").lower()

            if move == "go north":
                current_location = (current_location[0], current_location[1] + 1)
            elif move == "go south":
                current_location = (current_location[0], current_location[1] - 1)
            elif move == "go east":
                current_location = (current_location[0] + 1, current_location[1])
            elif move == "go west":
                current_location = (current_location[0] - 1, current_location[1])
            else:
                print("Invalid direction.")
                continue
            
            if current_location == self.boss_location:
                self.boss_fight(player)
                break
            else:
                self.random_encounter(player)

        print("Game Over. Returning to the main menu.")
    
    def boss_fight(self, player):
        boss = Enemy(name="Forest Boss", level=10, health=200, strength=15, defense=10)
        print("You have encountered the Boss!")
        self.battle(player, boss)
    
    def random_encounter(self, player):
        """Rencontrer des ennemis ou trouver des objets"""
        event = random.choice(["monster", "item", "nothing"])
        if event == "monster":
            enemy = Enemy(name="Goblin", level=1, health=50, strength=8, defense=3)
            print(f"A wild {enemy.name} appeared!")
            self.battle(player, enemy)
        elif event == "item":
            print("You found a potion!")
            player.inventory.potions += 1
        else:
            print("Nothing happens here.")

    def battle(self, player, enemy):
        """Mode combat au tour par tour"""
        while player.health > 0 and enemy.health > 0:
            print(f"{player.name}: {player.health} HP | {enemy.name}: {enemy.health} HP")
            action = input("Do you want to (1) Attack, (2) Use Potion, or (3) Run? ")

            if action == '1':
                damage = player.attack(enemy)
                enemy.health -= damage
                if enemy.health <= 0:
                    print(f"{enemy.name} has been defeated!")
                    player.gain_xp(50)
                    break
            elif action == '2':
                player.use_potion()
            elif action == '3':
                print(f"{player.name} fled the battle.")
                break
            else:
                print("Invalid action.")
                continue

            damage = enemy.attack(player)
            if not player.take_damage(damage):
                print(f"{player.name} has been defeated.")
                break
            
            
if __name__ == "__main__":
    game_main_menu.main()

MAIN MENU:
1. Create New Game
2. Load Saved Game
3. About
4. Exit
> 1
Creating New Game
Please Enter your name:
> Herison
Enter your age: 
> 15
Hello, Herison! Welcome to the game.
You are lost in the forest with only a knife.


TypeError: Player.__init__() got an unexpected keyword argument 'stength'

MAIN MENU:
1. Create New Game
2. Load Saved Game
3. About
4. Exit


KeyboardInterrupt: Interrupted by user

In [64]:
import random
import json

class GameMainMenu:
    @staticmethod
    def main_menu():
        print("MAIN MENU:")
        print("1. Create New Game")
        print("2. Load Saved Game")
        print("3. About")
        print("4. Exit")

    @staticmethod
    def create_new_game():
        print("Creating New Game")
        name = input("Please Enter your name: ")
        age = input("Enter your age: ")
        print(f"Hello, {name}! Welcome to the game.")
        
        player = Player(name=name, age=age, weapon="Knife", health=100, strength=10, defense=5, xp=0, gold=50)
        Game().start(player)
        player.choose_inventory()
        player.save_game()
        return player
    
    @staticmethod
    def load_saved_game():
        try:
            with open("savegame.json", "r") as file:
                data = json.load(file)
                player = Player(
                    name=data['name'],
                    age=data['age'],
                    weapon=data['weapon'],
                    health=data['health'],
                    strength=data['strength'],
                    defense=data['defense'],
                    xp=data['xp'],
                    gold=data['gold']
                )
                player.inventory.potions = data['inventory']['potions']
                player.inventory.attack_boost = data['inventory']['attack_boost']
                player.inventory.defense_boost = data['inventory']['defense_boost']
                print(f"Game Loaded! Welcome back, {player.name}.")
                player.inventory.show_inventory()
                return player
        except FileNotFoundError:
            print("No saved game found.")
            return None
        except json.JSONDecodeError:
            print("Error loading saved game. File may be corrupted.")
            return None
        
    @staticmethod
    def main():
        while True:
            GameMainMenu.main_menu()
            choice = input("> ")

            if choice == '1':
                GameMainMenu.create_new_game()
            elif choice == '2':
                player = GameMainMenu.load_saved_game()
                if player:
                    print(f"Loaded {player.name}'s game.")
            elif choice == '3':
                print("About - This is a simple text adventure game.")
            elif choice == '4':
                print("Exiting game. Goodbye!")
                break
            else:
                print("Invalid choice. Please select a valid option.")


class Entity:
    def __init__(self, name, health, strength, defense):
        self.name = name
        self.health = health
        self.strength = strength
        self.defense = defense

    
class Player(Entity):
    def __init__(self, name, age, weapon, health, strength, defense, xp, gold):
        super().__init__(name, health, strength, defense)
        self.age = age
        self.weapon = weapon
        self.xp = xp
        self.level = 1
        self.inventory = Inventory()
        self.gold = gold

    def level_up(self):
        required_xp = self.level * 100
        if self.xp >= required_xp:
            self.level += 1
            self.health += 20
            self.strength += 5
            self.defense += 3
            self.xp -= required_xp
            print(f"{self.name} leveled up to Level {self.level}!")

    def choose_inventory(self):
        print("Choose items for your inventory:")
        
        while True:
            print("\n1. Add Potion\n2. Add Attack Boost\n3. Add Defense Boost\n4. Show Inventory\n5. Exit")
            choice = input("> ")

            if choice == '1':
                amount = int(input("How many potions? > "))
                self.inventory.buy_potions(amount, self.gold)
            elif choice == '2':
                amount = int(input("How many attack boosts? > "))
                self.inventory.buy_attack_boost(amount, self.gold)
            elif choice == '3':
                amount = int(input("How many defense boosts? > "))
                self.inventory.buy_defense_boost(amount, self.gold)
            elif choice == '4':
                self.inventory.show_inventory()
            elif choice == '5':
                print("Exiting inventory selection.")
                break
            else:
                print("Invalid choice. Please select a valid option.")

    def attack(self, enemy):
        damage = max(0, self.strength - enemy.defense)
        print(f"{self.name} attacks {enemy.name} with {self.weapon} and causes {damage} damage.")
        return damage
    
    def take_damage(self, damage):
        self.health -= damage
        if self.health <= 0:
            print(f"{self.name} has been defeated.")
            return False
        return True

    def use_potion(self):
        if self.inventory.potions > 0:
            self.health += 30
            self.inventory.potions -= 1
            print(f"{self.name} used a potion. Health is now {self.health}.")
        else:
            print("No potions left!")
    
    def gain_xp(self, amount):
        self.xp += amount
        print(f"{self.name} gained {amount} XP.")
        self.level_up()
        
    def save_game(self):
        data = {
            'name': self.name,
            'age': self.age,
            'weapon': self.weapon,
            'health': self.health,
            'strength': self.strength,
            'defense': self.defense,
            'xp': self.xp,
            'gold': self.gold,
            'inventory': {
                'potions': self.inventory.potions,
                'attack_boost': self.inventory.attack_boost,
                'defense_boost': self.inventory.defense_boost
            }
        }
        with open("savegame.json", "w") as file:
            json.dump(data, file)
        print("Game saved successfully.")


class Enemy(Entity):
    def __init__(self, name, level, health, strength, defense):
        super().__init__(name, health, strength, defense)
        self.level = level

    def attack(self, player):
        damage = max(0, self.strength - player.defense)
        print(f"{self.name} attacks {player.name} and causes {damage} damage.")
        return damage


class Inventory:
    def __init__(self, potions=0, attack_boost=0, defense_boost=0):
        self.potions = potions
        self.attack_boost = attack_boost
        self.defense_boost = defense_boost

    def buy_potions(self, amount, gold):
        potion_cost = 10
        total_cost = amount * potion_cost

        if gold >= total_cost:
            self.potions += amount
            gold -= total_cost
            print(f"{amount} potion(s) purchased. Total: {self.potions}. Remaining gold: {gold}.")
        else:
            print("You don't have enough gold to buy these potions.")
        return gold

    def use_potion(self):
        if self.potions > 0:
            self.potions -= 1
            print(f"Potion used. Remaining: {self.potions}.")
        else:
            print("No potions available.")

    def buy_attack_boost(self, amount, gold):
        boost_cost = 20
        total_cost = amount * boost_cost

        if gold >= total_cost:
            self.attack_boost += amount
            gold -= total_cost
            print(f"{amount} attack boost(s) purchased. Total: {self.attack_boost}. Remaining gold: {gold}.")
        else:
            print("You don't have enough gold to buy these attack boosts.")
        return gold

    def buy_defense_boost(self, amount, gold):
        boost_cost = 20
        total_cost = amount * boost_cost

        if gold >= total_cost:
            self.defense_boost += amount
            gold -= total_cost
            print(f"{amount} defense boost(s) purchased. Total: {self.defense_boost}. Remaining gold: {gold}.")
        else:
            print("You don't have enough gold to buy these defense boosts.")
        return gold

    def show_inventory(self):
        print(f"Inventory:\nPotions: {self.potions}\nAttack Boosts: {self.attack_boost}\nDefense Boosts: {self.defense_boost}")


class Game:
    def __init__(self):
        self.map = {
            (0, 0): "Starting point",
            (1, 0): "A calm clearing",
            (0, 1): "A dark cave",
            (-1, 0): "A mysterious river",
            (0, -1): "A thick forest",
            (2, 0): "Boss Lair"
        }
        self.boss_location = (2, 0)
    
    def start(self, player):
        print("Game started!")
        current_location = (0, 0)
        while player.health > 0:
            print(f"Current location: {self.map.get(current_location, 'Unknown area')}")
            move = input("Where would you like to go? (Go North, Go South, Go East, Go West): ").lower()

            if move == "go north":
                current_location = (current_location[0], current_location[1] + 1)
            elif move == "go south":
                current_location = (current_location[0], current_location[1] - 1)
            elif move == "go east":
                current_location = (current_location[0] + 1, current_location[1])
            elif move == "go west":
                current_location = (current_location[0] - 1, current_location[1])
            else:
                print("Invalid direction.")
                continue
            
            if current_location == self.boss_location:
                self.boss_fight(player)
                break
            else:
                self.random_encounter(player)

        print("Game Over. Returning to the main menu.")
    
    def boss_fight(self, player):
        boss = Enemy(name="Forest Boss", level=10, health=200, strength=15, defense=10)
        print("You have encountered the Boss!")
        self.battle(player, boss)
    
    def random_encounter(self, player):
        """Rencontrer des ennemis ou trouver des objets"""
        event = random.choice(["monster", "item", "nothing"])
        if event == "monster":
            enemy = Enemy(name="Goblin", level=1, health=50, strength=8, defense=3)
            print(f"A wild {enemy.name} appeared!")
            self.battle(player, enemy)
        elif event == "item":
            print("You found a potion!")
            player.inventory.potions += 1
        else:
            print("Nothing happens here.")

    def battle(self, player, enemy):
        """Mode combat au tour par tour"""
        while player.health > 0 and enemy.health > 0:
            print(f"{player.name}: {player.health} HP | {enemy.name}: {enemy.health} HP")
            action = input("Do you want to (1) Attack, (2) Use Potion, or (3) Run? ")

            if action == '1':
                damage = player.attack(enemy)
                enemy.health -= damage
                if enemy.health <= 0:
                    print(f"{enemy.name} has been defeated!")
                    player.gain_xp(50)
                    break
            elif action == '2':
                player.use_potion()
            elif action == '3':
                print(f"{player.name} fled the battle.")
                break
            else:
                print("Invalid action.")
                continue

            damage = enemy.attack(player)
            if not player.take_damage(damage):
                print(f"{player.name} has been defeated.")
                break

if __name__ == "__main__":
    GameMainMenu.main()


MAIN MENU:
1. Create New Game
2. Load Saved Game
3. About
4. Exit
Creating New Game
Hello, Herison! Welcome to the game.
Game started!
Current location: Starting point
Nothing happens here.
Current location: A dark cave
A wild Goblin appeared!
Herison: 100 HP | Goblin: 50 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Herison and causes 3 damage.
Herison: 97 HP | Goblin: 43 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Herison and causes 3 damage.
Herison: 94 HP | Goblin: 36 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Herison and causes 3 damage.
Herison: 91 HP | Goblin: 29 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Herison and causes 3 damage.
Herison: 88 HP | Goblin: 22 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Herison and causes 3 damage.
Herison: 85 HP | Goblin: 15 HP
Herison attacks Goblin with Knife and causes 7 damage.
Goblin attacks Heri

KeyboardInterrupt: Interrupted by user